In [1]:
import cv2
from keras.models import load_model
import numpy as np
#queue
from collections import deque

Using TensorFlow backend.
C:\Users\super\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\super\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\super\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\super\AppData\Roa

# Loading trained Model

In [2]:
model1=load_model("C:\\Users\\super\\devanagari.h5")
print(model1)

W0923 09:58:03.337498  2676 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0923 09:58:03.361466  2676 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0923 09:58:03.381439  2676 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4267: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0923 09:58:03.429376  2676 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:190: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0923 09:58:03.433370  2676 deprecation_wrapper.py:119] From C:\ProgramDat

# Creating Dict with the alphabets

In [3]:
letter_count={0:'CHECK',1: '01_ka', 2: '02_kha', 3: '03_ga', 4: '04_gha', 5: '05_kna', 6: 'character_06_cha',
                    7: '07_chha', 8: '08_ja', 9: '09_jha', 10: '10_yna',
                    11: '11_taamatar',
                    12: '12_thaa', 13: '13_daa', 14: '14_dhaa', 15: '15_adna', 16: '16_tabala', 17: '17_tha',
                    18: '18_da',
                    19: '19_dha', 20: '20_na', 21: '21_pa', 22: '22_pha',
                    23: '23_ba',
                    24: '24_bha', 25: '25_ma', 26: '26_yaw', 27: '27_ra', 28: '28_la', 29: '29_waw', 30: '30_motosaw',
                    31: '31_petchiryakha',32: '32_patalosaw', 33: '33_ha',
                    34: '34_chhya', 35: '35_tra', 36: '36_gya',37:'CHECK'}

# Processing the images

In [4]:
def keras_predict(model,image):
    processed=keras_process_image(image)
    print("Processed:" + str(processed.shape))
    pred_probab=model.predict(processed)[0]
    pred_class=list(pred_probab).index(max(pred_probab))
    return max(pred_probab),pred_class

def keras_process_image(img):
    image_x=32
    image_y=32
    img=cv2.resize(img,(image_x,image_y))
    img=np.array(img,dtype=np.float32)
    img=np.reshape(img,(-1,image_x,image_y,1))
    return img

# Using your Camera

In [5]:
cap=cv2.VideoCapture(0)
Lower_blue=np.array([110,50,50])
Upper_blue=np.array([130,255,255])
pred_class=0
pts=deque(maxlen=512)
blackboard=np.zeros((480,640,3),dtype=np.uint8)
digit=np.zeros((200,200,3),dtype=np.uint8)
while(cap.isOpened()):
    ret,img=cap.read()
    img=cv2.flip(img,1)
    imgHSV=cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    mask=cv2.inRange(imgHSV,Lower_blue,Upper_blue)
    blur=cv2.medianBlur(mask,15)
    blur=cv2.GaussianBlur(blur,(5,5),0)
    thresh=cv2.threshold(blur,0,255,cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    cnts=cv2.findContours(thresh.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)[0]
    center=None
    if len(cnts) >= 1:
        contour=max(cnts,key=cv2.contourArea)
        if cv2.contourArea(contour)>250:
            ((x,y),radius)=cv2.minEnclosingCircle(contour)
            cv2.circle(img,(int(x),int(y)),int(radius),(0,255,255),2)
            cv2.circle(img,center,5,(0,0,255),-1)
            M=cv2.moments(contour)
            center=(int(M['m10']/M['m00']),int(M['m01']/M['m00']))
            pts.appendleft(center)
            for i in range(1,len(pts)):
                if pts[i-1] is None or pts[i] is None:
                    continue
                cv2.line(blackboard,pts[i-1],pts[i],(255,255,255),10)
                cv2.line(img,pts[i-1],pts[i],(0,0,255),5)
    elif len(cnts)==0:
        if len(pts)!=[]:
            blackboard_gray=cv2.cvtColor(blackboard,cv2.COLOR_BGR2GRAY)
            blur1=cv2.medianBlur(blackboard_gray,15)
            blur1=cv2.GaussianBlur(blur1,(5,5),0)
            thresh1=cv2.threshold(blur1,0,255,cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
            blackboard_cnts=cv2.findContours(thresh1.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)[0]
            if len(blackboard_cnts)>=1:
                cnt=max(blackboard_cnts,key=cv2.contourArea)
                print(cv2.contourArea(cnt))
                if cv2.contourArea(cnt)>2000:
                    x,y,w,h=cv2.boundingRect(cnt)
                    digit=blackboard_gray[y:y+h,x:x+w]
                    #newImage=process_letter(digit)
                    pred_probab,pred_class=keras_predict(model1,digit)
                    print(pred_class,pred_probab)
            
        pts=deque(maxlen=512)
        blackboard=np.zeros((480,640,3),dtype=np.uint8)
    cv2.putText(img,"Conv Network:" + str(letter_count[pred_class]),(10,470),
                cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,0,255),2)
    cv2.imshow("Frame",img)
    cv2.imshow("Contours",thresh)
    k=cv2.waitKey(10)
    if k==27:
        break

2301.0
Processed:(1, 32, 32, 1)
34 1.0
16865.5
Processed:(1, 32, 32, 1)
26 1.0
16596.0
Processed:(1, 32, 32, 1)
10 1.0
8611.5
Processed:(1, 32, 32, 1)
1 1.0
9894.0
Processed:(1, 32, 32, 1)
32 1.0
13873.0
Processed:(1, 32, 32, 1)
34 1.0
743.5
522.0
7634.5
Processed:(1, 32, 32, 1)
25 1.0
3078.5
Processed:(1, 32, 32, 1)
36 1.0
4264.5
Processed:(1, 32, 32, 1)
2 1.0
35.5
4620.5
Processed:(1, 32, 32, 1)
9 1.0
279.0
7990.5
Processed:(1, 32, 32, 1)
31 1.0
1886.5
5864.0
Processed:(1, 32, 32, 1)
15 1.0
13057.0
Processed:(1, 32, 32, 1)
2 1.0
4703.5
Processed:(1, 32, 32, 1)
25 1.0
2115.5
Processed:(1, 32, 32, 1)
16 1.0
1076.0
18482.0
Processed:(1, 32, 32, 1)
29 1.0
702.0
11100.0
Processed:(1, 32, 32, 1)
23 1.0
6942.0
Processed:(1, 32, 32, 1)
28 1.0
99.0
2994.0
Processed:(1, 32, 32, 1)
20 1.0
4487.5
Processed:(1, 32, 32, 1)
20 1.0
6458.5
Processed:(1, 32, 32, 1)
36 1.0
4503.5
Processed:(1, 32, 32, 1)
34 1.0
6429.5
Processed:(1, 32, 32, 1)
32 1.0
5797.0
Processed:(1, 32, 32, 1)
16 1.0
85.0
3344.0
Pr

KeyboardInterrupt: 